In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.nddata import CCDData

In [ ]:
import numpy as np


In [ ]:
import sys
sys.path.append("../imaging")
sys.path.append("../photometry")
from phot_utils import xmatch

In [ ]:
import tomllib

In [ ]:
import arya

In [ ]:
import read_iso

In [ ]:
import seaborn as sns

In [ ]:
import shapely

In [ ]:
obj = "yasone1"

In [ ]:
if obj == "yasone1":
    ra0, dec0 = 265.52019, 13.17146
elif obj == "yasone2":
    ra0, dec0 = 262.34921, 6.42302
elif obj == "yasone3":
    ra0, dec0 = 292.96258, -26.44994

In [ ]:
def get_mag_shift(objname):
    shifts = {}
    with open("../photometry/" + objname + "/panstarrs_shift.toml", "rb") as f:
        shifts = tomllib.load(f)
    return shifts

In [ ]:
def read_catalogue(objname):
    cat = Table.read(f"../photometry/{objname}/allcolours.cat")

    shifts = get_mag_shift(objname)
    cat["GMAG"] -= shifts["g"]
    cat["RMAG"] -= shifts["r"]
    cat["IMAG"] -= shifts["i"]
    cat = cat[(cat["RFLAGS"] < 8 ) & (cat["RFLAGS_WEIGHT"] == 0) ]
    return cat

In [ ]:
def read_catalogue_julen(objname):
    cat = Table.read(f"{objname}_julen.cat")

    return cat

In [ ]:
get_mag_shift(obj)

In [ ]:
cat = read_catalogue(obj)

In [ ]:
cat_julen = read_catalogue_julen(obj)

In [ ]:
from convien

In [ ]:
xmatch_idx, xmatch_dist, xmatch_filt = xmatch(cat["ALPHA_J2000"], cat["DELTA_J2000"], cat_julen["ALPHA_J2000"], cat_julen["DELTA_J2000"])

In [ ]:
plt.scatter(cat["ALPHA_J2000"], cat["DELTA_J2000"], s=2)
plt.scatter(cat_julen["ALPHA_J2000"], cat_julen["DELTA_J2000"], s=1)


In [ ]:
plt.scatter(cat["ALPHA_J2000"][xmatch_filt], cat_julen["ALPHA_J2000"][xmatch_idx[xmatch_filt]] - cat["ALPHA_J2000"][xmatch_filt])
plt.scatter(cat["ALPHA_J2000"][xmatch_filt], cat_julen["DELTA_J2000"][xmatch_idx[xmatch_filt]] - cat["DELTA_J2000"][xmatch_filt])

In [ ]:
4*1.7

In [ ]:
np.median(cat_julen["fwhm"]) * 1.7

In [ ]:
np.sqrt(7.3**2 * np.pi) * 33

In [ ]:
plt.scatter(cat["FWHM_IMAGE"][xmatch_filt], cat_julen["fwhm"][xmatch_idx[xmatch_filt]], s=1, alpha=0.3)
# plt.xscale("log")
# plt.yscale("log")

plt.plot([0, 10], [0, 10], color="k")
plt.xlim(0, 10)
plt.ylim(0, 10)

In [ ]:
 426 / cat["FLUX_APER"][:, 3]

In [ ]:
-cat_julen["RMAG"]+36

In [ ]:
# plt.scatter(cat["RMAG"], cat["RMAG_ERR"] , s=1, color=arya.COLORS[0], label="me")
bkg_err = 500
# plt.scatter(cat["RMAG"], 0.02 + bkg_err / cat["FLUX_APER"][:, 1] * 2.5 / np.log(10) , s=1, color=arya.COLORS[0], label="me")
plt.scatter(cat["RMAG"], 0.01 + 15 / cat["FLUX_APER"][:, 4]**0.5 * 2.5 / np.log(10) , s=1, color=arya.COLORS[0], label="15x poisson (me)")
plt.scatter(cat_julen["RMAG"], cat_julen["RMAG_ERR"], s=1, color=arya.COLORS[1], label="julen")
plt.scatter(cat_julen["RMAG"], 0.01 / np.sqrt(10**(0.4*(-cat_julen["RMAG"]+18))), s=0.1, color=arya.COLORS[2], label="rescaled poisson error (julen)")
plt.legend()
plt.ylim(0, 1)
plt.xlim(15, 28)
plt.xlabel("r mag")
plt.ylabel("r mag err")
plt.axhline(0.2)

In [ ]:
factor = 3
plt.scatter(cat["GMAG_ERR"][xmatch_filt], cat_julen["GMAG_ERR"][xmatch_idx[xmatch_filt]] - cat["GMAG_ERR"][xmatch_filt] * factor, s=1, color=arya.COLORS[2], label="g")


plt.scatter(cat["RMAG_ERR"][xmatch_filt], cat_julen["RMAG_ERR"][xmatch_idx[xmatch_filt]] - cat["RMAG_ERR"][xmatch_filt] * factor, s=1, color=arya.COLORS[3], label="r")


plt.scatter(cat["IMAG_ERR"][xmatch_filt], cat_julen["IMAG_ERR"][xmatch_idx[xmatch_filt]] - cat["IMAG_ERR"][xmatch_filt] * factor, s=1, color=arya.COLORS[1], label="i")

plt.axhline(0, color="black")

plt.xlabel("magnitude")
plt.ylabel("mag err residual (julen - me)")
plt.ylim(-0.1, 0.3)
arya.Legend(-1)

In [ ]:
plt.scatter(cat["GMAG"][xmatch_filt], cat_julen["GMAG"][xmatch_idx[xmatch_filt]] - cat["GMAG"][xmatch_filt], s=1, color=arya.COLORS[2], label="g")


plt.scatter(cat["RMAG"][xmatch_filt], cat_julen["RMAG"][xmatch_idx[xmatch_filt]] - cat["RMAG"][xmatch_filt], s=1, color=arya.COLORS[3], label="r")


plt.scatter(cat["IMAG"][xmatch_filt], cat_julen["IMAG"][xmatch_idx[xmatch_filt]] - cat["IMAG"][xmatch_filt], s=1, color=arya.COLORS[1], label="i")

plt.axhline(0, color="black")

plt.xlabel("magnitude")
plt.ylabel("residual (julen - me)")
plt.ylim(-1, 0.5)
arya.Legend(-1)

In [ ]:
plt.scatter(cat["RMAG"], cat["MAG_AUTO"] - cat["RMAG"], s=1, alpha=0.05)

In [ ]:
plt.scatter(cat["RMAG"], cat["MAG_AUTO"] - cat["RMAG"], s=1, alpha=0.05)

In [ ]:
plt.scatter(cat["RMAG"], cat["MAG_ISO"] - cat["RMAG"], s=1, alpha=0.05)

In [ ]:
plt.scatter(cat["RMAG"], cat["MAG_WIN"] - cat["RMAG"], s=1, alpha=0.05)